### 1. Import modules

In [1]:
import os
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
nltk.download('punkt')    # For tokenization
nltk.download('averaged_perceptron_tagger')  # For POS tagging

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vanng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vanng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vanng\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### 2. Load data

In [2]:
train_path = "../datasets/train"
categories = ['neg', 'pos']
data = []

def is_git_lfs_file(file_path):
    """
    Check if a file is a Git LFS pointer
    """
    with open(file_path, "r", encoding="utf-8") as file:
        first_line = file.readline().strip()
        return first_line.startswith("version https://git-lfs.github.com")

for category in categories:
    category_path = os.path.join(train_path, category)
    category= 0 if category == "neg" else 1

    for filename in os.listdir(category_path):
        file_path = os.path.join(category_path, filename)

        if is_git_lfs_file(file_path): # skip Git LFS pointer files
            continue

        with open(file_path, "r", encoding='utf-8') as file:
            content = file.read().strip()
        
        data.append((filename, content, category))

df = pd.DataFrame(data, columns=['filename', 'content', 'category'])

df

,filename,content,category
0,10000_4.txt,Airport '77 starts as a brand new luxury 747 p...,0
1,10006_4.txt,"I don't know who to blame, the timid writers o...",0
2,10009_1.txt,This film is one giant pant load. Paul Schrade...,0
3,1000_4.txt,"The plot for Descent, if it actually can be ca...",0
4,10017_4.txt,"""Ghost of Dragstrip Hollow"" appears to take pl...",0
...,...,...,...
1120,10896_8.txt,At first i didn't think that Ben Affleck could...,1
1121,10897_9.txt,What would you expect from a film titled 'Surv...,1
1122,10898_7.txt,This movie isn't as bad as I heard. It was enj...,1
1123,10899_10.txt,I laughed so hard during this movie my face hu...,1


### 3. Transform data


#### 3.1. Convert data from RAW to Tokens

In [6]:
def convert_tokens(rawtext, verbose):
    # 1. Tokenization
    pattern = r'\w+'
    tokenizer = RegexpTokenizer(pattern)
    token_words = tokenizer.tokenize(rawtext)
    if (verbose):
        print('Tokens:'+str(token_words[0:10]))
    
    # 2. Decapitalization
    decap_token_words = [word.lower() for word in token_words]
    if (verbose):
        print("Decapitalized tokens:" + str(decap_token_words[0:10]))
    
    # 3. Remove stop words
    stopwords_nltk_en = set(stopwords.words('english'))
    rmsw_token_words = ([word for word in token_words if word.lower() not in stopwords_nltk_en])
    if (verbose):
        print('Stopwords removed:' + str(rmsw_token_words[0:20]))
    
    # 4. Remove CAP words
    rmcap_token_words = []
    for word in rmsw_token_words:
        if word.isupper():
            rmcap_token_words.append(word.title())
        else:
            rmcap_token_words.append(word)
    if (verbose):
        print('CAPITALIZED removed:' + str(rmcap_token_words[0:20]))
    
    # 5. Remove salutation
    salutation = ['mr', 'mrs', 'ms', 'dr', 'phd', 'prof', 'rev']
    rmsalu_token_words = ([word for word in rmsw_token_words if word.lower() not in salutation])
    if (verbose):
        print('Salutation removed:' + str(rmsalu_token_words[0:20]))
    
    # 6. Define transfer tag function:
    def transfer_tag(treebank_tag):
        if treebank_tag.startswith('j') or treebank_tag.startswith('J'):
            return "a"
        elif treebank_tag.startswith('v') or treebank_tag.startswith('V'):
            return "v"
        elif treebank_tag.startswith('n') or treebank_tag.startswith('N'):
            return 'n'
        elif treebank_tag.startswith('r') or treebank_tag.startswith('R'):
            return 'r'
        else:
            return 'n'
    
    # 7. Lemmatization
    wnl = WordNetLemmatizer()

    lemma_words = []
    for word, tag in nltk.pos_tag(rmsalu_token_words):
        firstletter = tag[0].lower()
        wtag = transfer_tag(firstletter)
        if not wtag:
            lemma_words.extend([word])
        else:
            lemma_words.extend([wnl.lemmatize(word, wtag)])
    if verbose:
        print('Lemma:' + str(lemma_words[0:10]))
    
    # 8. English words
    eng_words = [word for word in lemma_words if len(wn.synsets(word.lower())) > 1]

    # 9 Remove numbers
    rmnb_token_words = ([word for word in eng_words if not word.isdigit()])
    if (verbose):
        print('Number removed:' + str(rmnb_token_words[0:20]))
    
    return rmnb_token_words

In [7]:
df_handle = df.copy()
[n, d] = df_handle.shape
df_handle['tokens'] = ['']*n

for index, row in df_handle.iterrows():
    df_handle['tokens'].iloc[index] = convert_tokens(row['content'], verbose = False)

df_handle.head(10)

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - 'C:\\Users\\vanng/nltk_data'
    - 'c:\\Users\\vanng\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\vanng\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\vanng\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\vanng\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
